# GPUStack API の使用例

このノートブックでは、GPUStackのOpenAI互換APIを使用して、ローカルにデプロイされたLLMモデルとやり取りする基本的な方法を紹介します。

## 1. 必要なライブラリのインポート

In [ ]:
import os
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
from openai import OpenAI
from dotenv import load_dotenv

## 2. GPUStack APIの設定

In [ ]:
# .envファイルがある場合は読み込む
load_dotenv()

# GPUStackのAPI設定
GPUSTACK_API_BASE = os.getenv("GPUSTACK_API_BASE", "http://localhost:8000/v1")
GPUSTACK_API_KEY = os.getenv("GPUSTACK_API_KEY", "")

# APIキーが設定されていない場合は、手動で入力することもできます
# from getpass import getpass
# GPUSTACK_API_KEY = getpass("GPUStackのAPIキーを入力してください: ")

## 3. GPUStackが実行中かどうかを確認

In [ ]:
def check_gpustack_running():
    """GPUStackが実行中かどうかを確認する"""
    try:
        response = requests.get(f"{GPUSTACK_API_BASE}/models")
        return response.status_code == 200
    except requests.exceptions.ConnectionError:
        return False

if check_gpustack_running():
    print("✅ GPUStackサーバーに正常に接続できました")
else:
    print("❌ GPUStackサーバーに接続できません。サーバーが実行中であることを確認してください。")
    print("   以下のコマンドを実行してGPUStackを起動してください: gpustack start")

## 4. デプロイされているモデルのリストを取得

In [ ]:
def get_models():
    """デプロイされているモデルのリストを取得する"""
    headers = {}
    if GPUSTACK_API_KEY:
        headers["Authorization"] = f"Bearer {GPUSTACK_API_KEY}"
    
    try:
        response = requests.get(f"{GPUSTACK_API_BASE}/models", headers=headers)
        if response.status_code == 200:
            return response.json()["data"]
        return []
    except:
        return []

# デプロイされているモデルを取得
models = get_models()

# モデル情報をDataFrameで表示
if models:
    models_df = pd.DataFrame(models)
    models_df[['id', 'status', 'type', 'device']]
else:
    print("デプロイされているモデルがありません。モデルをデプロイしてください。")

## 5. OpenAI互換APIを使用してLLMと対話する

In [ ]:
# 実行中のモデルを選択
running_models = [model for model in models if model["status"] == "RUNNING"]

if not running_models:
    print("実行中のモデルがありません。モデルが起動するのを待つか、新しいモデルをデプロイしてください。")
else:
    # 最初の実行中モデルを使用
    model_id = running_models[0]["id"]
    print(f"以下のモデルを使用します: {model_id}")

In [ ]:
# OpenAI互換クライアントの作成
client = OpenAI(
    api_key=GPUSTACK_API_KEY or "dummy_key",  # APIキーがない場合はダミーの値を使用
    base_url=GPUSTACK_API_BASE
)

# モデルとチャットする関数
def chat_with_model(prompt, model=model_id, max_tokens=500, temperature=0.7):
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "あなたは役立つAIアシスタントです。"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"

# テストプロンプト
test_prompt = "福岡の有名な観光スポットを3つ教えてください。それぞれについて簡単な説明も加えてください。"

# モデルから応答を取得
response = chat_with_model(test_prompt)
print(response)

## 6. パラメータを変更してみる

In [ ]:
# 創造性の高い応答（temperature=1.0）
creative_response = chat_with_model(
    "福岡を舞台にしたショートストーリーを書いてください。100単語程度で簡潔に。",
    temperature=1.0
)
print("Temperature=1.0 (創造的な応答):")
print("-" * 80)
print(creative_response)
print("-" * 80)

# より決定論的な応答（temperature=0.2）
deterministic_response = chat_with_model(
    "福岡を舞台にしたショートストーリーを書いてください。100単語程度で簡潔に。",
    temperature=0.2
)
print("\nTemperature=0.2 (決定論的な応答):")
print("-" * 80)
print(deterministic_response)
print("-" * 80)

## 7. チャット履歴を使用した会話

In [ ]:
def chat_with_history(messages, model=model_id, max_tokens=500, temperature=0.7):
    """チャット履歴を使用してモデルと対話する"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"

# 会話履歴の初期化
conversation = [
    {"role": "system", "content": "あなたは親切なガイドです。福岡について詳しく、地元の人しか知らないような情報も教えてくれます。"}
]

# 最初の質問
user_message = "福岡に初めて旅行します。おすすめの季節はいつですか？"
conversation.append({"role": "user", "content": user_message})
print(f"ユーザー: {user_message}")

# モデルの応答
assistant_response = chat_with_history(conversation)
conversation.append({"role": "assistant", "content": assistant_response})
print(f"アシスタント: {assistant_response}")

# フォローアップの質問
user_message = "その季節に開催される地元のお祭りや行事はありますか？"
conversation.append({"role": "user", "content": user_message})
print(f"\nユーザー: {user_message}")

# モデルの応答
assistant_response = chat_with_history(conversation)
conversation.append({"role": "assistant", "content": assistant_response})
print(f"アシスタント: {assistant_response}")

## 8. JSONモードでの出力

In [ ]:
def chat_with_json_mode(prompt, model=model_id, max_tokens=500, temperature=0.7):
    """JSON形式で応答を取得する"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": "あなたはJSON形式で応答するアシスタントです。"},
                {"role": "user", "content": prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature,
            response_format={"type": "json_object"}
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"

# JSONフォーマットでデータを要求
json_prompt = "福岡の有名な3つのラーメン店について、店名、場所、特徴をJSON形式で教えてください。"
json_response = chat_with_json_mode(json_prompt)

# 応答を表示
print(json_response)

# JSONをパースしてDataFrameで表示
try:
    data = json.loads(json_response)
    if "ramen_shops" in data:
        pd.DataFrame(data["ramen_shops"])
    else:
        for key in data.keys():
            if isinstance(data[key], list):
                pd.DataFrame(data[key])
                break
except Exception as e:
    print(f"JSONのパースに失敗しました: {e}")

## 9. 応答時間とトークン数の測定

In [ ]:
import time

# 異なる長さのプロンプトをテスト
prompts = [
    "こんにちは",
    "福岡の特産品について教えてください",
    "福岡の歴史、文化、観光スポット、グルメ、交通について詳しく教えてください。また、福岡に旅行する際のおすすめプランも紹介してください。"
]

results = []

for prompt in prompts:
    # 応答時間を測定
    start_time = time.time()
    response = chat_with_model(prompt, max_tokens=500)
    end_time = time.time()
    
    # 結果を記録
    results.append({
        "prompt": prompt,
        "prompt_length": len(prompt),
        "response_length": len(response),
        "time": end_time - start_time
    })

# 結果をDataFrameで表示
results_df = pd.DataFrame(results)
results_df

# 応答時間のグラフを表示
plt.figure(figsize=(10, 5))
plt.bar(results_df["prompt_length"], results_df["time"])
plt.xlabel("プロンプトの長さ（文字数）")
plt.ylabel("応答時間（秒）")
plt.title("プロンプトの長さと応答時間の関係")
plt.grid(axis="y")
plt.show()

## 10. システムプロンプトの効果

In [ ]:
def chat_with_system_prompt(system_prompt, user_prompt, model=model_id, max_tokens=500, temperature=0.7):
    """異なるシステムプロンプトでの応答を比較する"""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            max_tokens=max_tokens,
            temperature=temperature
        )
        return response.choices[0].message.content
    except Exception as e:
        return f"エラーが発生しました: {str(e)}"

# テスト用の質問
test_question = "福岡について教えてください"

# 異なるシステムプロンプト
system_prompts = [
    "あなたは簡潔に答えるアシスタントです。短く、要点だけを伝えてください。",
    "あなたは詳しく説明するアシスタントです。詳細な情報と背景知識も含めて回答してください。",
    "あなたは地元の福岡人になりきって会話してください。方言も使い、親しみやすく話してください。"
]

# 各システムプロンプトでの応答を取得
for i, system_prompt in enumerate(system_prompts, 1):
    response = chat_with_system_prompt(system_prompt, test_question)
    print(f"システムプロンプト {i}:\n{system_prompt}")
    print("\n応答:")
    print("-" * 80)
    print(response)
    print("-" * 80)
    print()

## 11. まとめ

このノートブックでは、GPUStackのOpenAI互換APIを使用して、ローカルにデプロイされたLLMモデルを様々な方法で活用する方法を紹介しました。これらの手法を応用して、独自のアプリケーションやサービスを構築することができます。

主なポイント：
1. OpenAI互換APIを使ってローカルのLLMにアクセスできる
2. チャット履歴を用いた会話の実現
3. 異なるパラメータ設定による応答の変化
4. システムプロンプトによる応答スタイルの制御
5. JSON形式での構造化データの取得

詳細については、[GPUStack公式ドキュメント](https://docs.gpustack.ai/)を参照してください。